In [11]:
import numpy as np


- how to train/test split
- windowinf
- preidctions
- evaluatiopn



### Windowing dataset


We've got to window our time series.

Why do we window?

Windowing is a method to turn a time series dataset into a **supervised learning problem**. 

In other words, we want to use windows of the past to predict the future.

For example for a univariate time series, windowing for one week (`window=7`) to predict the next single value (`horizon=1`) might look like:

```
Window for one week (univariate time series)

[0, 1, 2, 3, 4, 5, 6] -> [7]
[1, 2, 3, 4, 5, 6, 7] -> [8]
[2, 3, 4, 5, 6, 7, 8] -> [9]
```

Or for the price of Bitcoin, it'd look like:

```
Window for one week with the target of predicting the next day (Bitcoin prices)

[123.654, 125.455, 108.584, 118.674, 121.338, 120.655, 121.795] -> [123.033]
[125.455, 108.584, 118.674, 121.338, 120.655, 121.795, 123.033] -> [124.049]
[108.584, 118.674, 121.338, 120.655, 121.795, 123.033, 124.049] -> [125.961]
```

Let's build some functions which take in a univariate time series and turn it into windows and horizons of specified sizes.

Now we'll write a function to take in an array and turn it into a window and horizon.

In [1]:
# Create function to label windowed data
def get_labelled_windows(x, horizon=1):
  """
  Creates labels for windowed dataset.

  E.g. if horizon=1 (default)
  Input: [1, 2, 3, 4, 5, 6] -> Output: ([1, 2, 3, 4, 5], [6])
  """
  return x[:, :-horizon], x[:, -horizon:]

In [3]:
# Test out the window labelling function with window_size=7 and horizon=1
test_window, test_label = get_labelled_windows(tf.expand_dims(tf.range(8)+1, axis=0), horizon=1)
print(f"Window: {tf.squeeze(test_window).numpy()} -> Label: {tf.squeeze(test_label).numpy()}")

NameError: name 'tf' is not defined

Now we need a way to make windows for an entire time series.

We could do this with Python for loops, however, for large time series, that'd be quite slow.

To speed things up, we'll leverage [NumPy's array indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html).

Let's write a function which:
1. Creates a window step of specific window size, for example: `[[0, 1, 2, 3, 4, 5, 6, 7]]`
2. Uses NumPy indexing to create a 2D of multiple window steps, for example: 
```
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [2, 3, 4, 5, 6, 7, 8, 9]]
```
3. Uses the 2D array of multuple window steps to index on a target series
4. Uses the `get_labelled_windows()` function we created above to turn the window steps into windows with a specified horizon

> 📖 **Resource:** The function created below has been adapted from Syafiq Kamarul Azman's article [*Fast and Robust Sliding Window Vectorization with NumPy*](https://towardsdatascience.com/fast-and-robust-sliding-window-vectorization-with-numpy-3ad950ed62f5).

In [4]:
# Create function to view NumPy arrays as windows 
def make_windows(x, window_size=7, horizon=1):
    """
    Turns a 1D array into a 2D array of sequential windows of window_size.
    """
    # 1. Create a window of specific window_size (add the horizon on the end for later labelling)
    window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
    # print(f"Window step:\n {window_step}")

    # 2. Create a 2D array of multiple window steps (minus 1 to account for 0 indexing)
    window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).T # create 2D array of windows of size window_size
    # print(f"Window indexes:\n {window_indexes[:3], window_indexes[-3:], window_indexes.shape}")

    # 3. Index on the target array (time series) with 2D array of multiple window steps
    windowed_array = x[window_indexes]

    # 4. Get the labelled windows
    windows, labels = get_labelled_windows(windowed_array, horizon=horizon)

    return windows, labels

> 🔑 **Note:** You can find a function which achieves similar results to the ones we implemented above at [`tf.keras.preprocessing.timeseries_dataset_from_array()`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/timeseries_dataset_from_array). Just like ours, it takes in an array and returns a windowed dataset. It has the benefit of returning data in the form of a tf.data.Dataset instance (we'll see how to do this with our own data later).

## PREDS

In [5]:
# wrong way!!!
# This way of making predictions is not correct, but it can be useful for a quick comparison of the models.
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model 
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data)
  return tf.squeeze(forecast) # return 1D array of predictions

------------DIBUJO

<img src='images/preds.png'/>
*Example flow chart representing the loop we're about to create for making forecasts. Not pictured: retraining a forecasting model every time a forecast is made & new data is acquired. For example, if you're predicting the price of Bitcoin daily, you'd want to retrain your model every day, since each day you're going to have a new data point to work with.*

Alright, let's create a function which returns `INTO_FUTURE` forecasted values using a trained model.

To do so, we'll build the following steps:
1. Function which takes as input: 
  * a list of values (the Bitcoin historical data)
  * a trained model (such as `model_9`)
  * a window into the future to predict (our `INTO_FUTURE` variable)
  * the window size a model was trained on (`WINDOW_SIZE`) - the model can only predict on the same kind of data it was trained on
2. Creates an empty list for future forecasts (this will be returned at the end of the function) and extracts the last `WINDOW_SIZE` values from the input values (predictions will start from the last `WINDOW_SIZE` values of the training data)
3. Loop `INTO_FUTURE` times making a prediction on `WINDOW_SIZE` datasets which update to remove the first the value and append the latest prediction 
  * Eventually future predictions will be made using the model's own previous predictions as input.
  

  

In [14]:
def make_future_forecast(model, initial_window, into_future, window_size=32, verbose=False) -> np.ndarray:
    """
    Generate future predictions using a sliding window approach.

    Parameters:
    - model: A trained TensorFlow/Keras model that takes input of shape (1, window_size) and outputs one prediction.
    - initial_window (np.ndarray): The initial time window to start predictions from (length = window_size).
    - into_future (int): Number of future time steps to predict.
    - window_size (int): Size of the input window for the model. Default is G.WINDOW_SIZE.
    - verbose (bool): If True, prints each prediction step.

    Returns:
    - np.ndarray: Array of predicted values with shape (into_future,).
    """
    future_forecast = []
    last_window = initial_window

    for step in range(into_future):
        future_pred = model.predict(tf.expand_dims(last_window, axis=0), verbose=0)

        if verbose:
            print(f"Predicting on: \n {last_window} -> Prediction: {tf.squeeze(future_pred).numpy()}\n")

        future_forecast.append(tf.squeeze(future_pred).numpy())
        last_window = np.append(last_window, future_pred[0][0])[-window_size:]

    return np.array(future_forecast)


In [13]:
# for multi-step forecasting (horizon > 1)

def make_future_forecast(model, initial_window, into_future, horizon, window_size=32, verbose=False) -> np.ndarray:
    """
    Generate future predictions using a sliding window for models that output multiple steps at once.

    Parameters:
    - model: A trained TensorFlow/Keras model that takes input of shape (1, window_size) and outputs (1, horizon).
    - initial_window (np.ndarray): The initial time window to start predictions from (length = window_size).
    - into_future (int): Total number of time steps to forecast (not number of prediction rounds).
    - horizon (int): Number of steps the model predicts at each call.
    - window_size (int): Size of the input window for the model.
    - verbose (bool): If True, prints each prediction step.

    Returns:
    - np.ndarray: Array of shape (into_future,) with all predictions concatenated and trimmed to match the exact horizon.
    """
    future_forecast = []
    last_window = initial_window.copy()
    
    steps = int(np.ceil(into_future / horizon))

    for _ in range(steps):
        future_pred = model.predict(tf.expand_dims(last_window, axis=0), verbose=0)  # shape: (1, horizon)
        future_pred = tf.squeeze(future_pred).numpy()  # shape: (horizon,)
        
        if verbose:
            print(f"Predicting on: \n{last_window} -> Prediction: {future_pred}\n")

        future_forecast.extend(future_pred.tolist())

        # Slide the window forward by appending the predictions
        last_window = np.append(last_window, future_pred)[-window_size:]

    return np.array(future_forecast[:into_future])


### Evaluation

In [17]:
# MASE implemented courtesy of sktime - https://github.com/alan-turing-institute/sktime/blob/ee7a06843a44f4aaec7582d847e36073a9ab0566/sktime/performance_metrics/forecasting/_functions.py#L16
def mean_absolute_scaled_error(y_true, y_pred):
  """
  Implement MASE (assuming no seasonality of data).
  """
  mae = tf.reduce_mean(tf.abs(y_true - y_pred))

  # Find MAE of naive forecast (no seasonality)
  mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:] - y_true[:-1])) # our seasonality is 1 day (hence the shifting of 1 day)

  return mae / mae_naive_no_season



# single horizon
def evaluate_preds(y_true, y_pred):
    # Make sure float32 (for metric calculations)
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    # Calculate various metrics
    mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
    mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) # puts and emphasis on outliers (all errors get squared)
    rmse = tf.sqrt(mse)
    mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
    mase = mean_absolute_scaled_error(y_true, y_pred)

    return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy(),
          "mase": mase.numpy()}


# multi-step horizon
def evaluate_preds_generalized(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
  mase = mean_absolute_scaled_error(y_true, y_pred)

  # Account for different sized metrics (for longer horizons, reduce to single number)
  if mae.ndim > 0: # if mae isn't already a scalar, reduce it to one by aggregating tensors to mean
    mae = tf.reduce_mean(mae)
    mse = tf.reduce_mean(mse)
    rmse = tf.reduce_mean(rmse)
    mape = tf.reduce_mean(mape)
    mase = tf.reduce_mean(mase)

  return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy(),
          "mase": mase.numpy()}



# better!!!!!!!!!!!!!!!!!1
def evaluate_preds(y_true, y_pred):
    """
    Evaluate forecast predictions using common regression metrics.
    Works with both single-step and multi-step predictions. Automatically
    reduces multi-step metrics to scalars by averaging.

    Parameters:
    - y_true (array-like): Ground truth values (1D or 2D).
    - y_pred (array-like): Predicted values (same shape as y_true).

    Returns:
    - dict: Dictionary with MAE, MSE, RMSE, MAPE, and MASE scores.
    """
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    mae = tf.reduce_mean(tf.keras.metrics.mean_absolute_error(y_true, y_pred))
    mse = tf.reduce_mean(tf.keras.metrics.mean_squared_error(y_true, y_pred))
    rmse = tf.sqrt(mse)
    mape = tf.reduce_mean(tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred))
    mase = tf.reduce_mean(mean_absolute_scaled_error(y_true, y_pred))

    return {
        "mae": mae.numpy(),
        "mse": mse.numpy(),
        "rmse": rmse.numpy(),
        "mape": mape.numpy(),
        "mase": mase.numpy()
    }
       



Yes, your function is appropriate for evaluating multi-step forecasts (i.e., horizon > 1), as long as:

y_true and y_pred have the same shape, e.g., (num_samples, horizon) or just (horizon,).

The model outputs multiple steps at once (direct multi-step), and you're evaluating the whole sequence against the ground truth.

✅ Why It Works for horizon > 1
TensorFlow's built-in metrics like mean_absolute_error return a vector when comparing (batch_size, horizon) arrays — one MAE per time step.

You correctly reduce them with tf.reduce_mean(...) to get a single scalar per metric.

Your fallback to .numpy() is fine for returning usable values in Python.

✅ Good Design Choices
✅ Type casting to float32

✅ Handles both scalar and vector metrics

✅ General enough for both single- and multi-step evaluation

✅ Custom mase function is included — good for time series

⚠️ Minor Suggestions
If your data is 1D (no batch dimension), e.g. y_true.shape == (horizon,), and you're comparing it with y_pred.shape == (horizon,), it still works — but you could clarify this in the docstring (if added later).

If your mean_absolute_scaled_error() function expects sequences too, make sure it's compatible with multi-step input.

Let me know if you want to:

Add support for per-time-step metric output (e.g., error at step 1, step 2, etc.)

Add a docstring

Support batched predictions (e.g., shape = (N, horizon))

✅ Why the Test Set is a “Pseudofuture”
In time series, you simulate the forecasting task by splitting historical data into:

Training set: Past data used to train the model.

Test set: Later part of the historical data, treated as if it's the future.

This test set lets you evaluate how your model might behave on future unseen data, but:

It's not the actual future — the data already exists.

Your model may perform differently when deployed, especially if real-world conditions change (called data drift or concept drift).


> 📖 **Resource:** Working with time series data can be tricky compared to other kinds of data. And there are a few pitfalls to watch out for, such as how much data to use for a test set. The article [*3 facts about time series forecasting that surprise experienced machine learning practitioners*](https://towardsdatascience.com/3-facts-about-time-series-forecasting-that-surprise-experienced-machine-learning-practitioners-69c18ee89387) talks about different things to watch out for when working with time series data.

Before we discuss what modelling experiments we're going to run, there are two terms you should be familiar with, **horizon** and **window**. 
  * **horizon** = number of timesteps to predict into future
  * **window** = number of timesteps from past used to predict **horizon**

For example, if we wanted to predict the price of Bitcoin for tomorrow (1 day in the future) using the previous week's worth of Bitcoin prices (7 days in the past), the horizon would be 1 and the window would be 7.

One of the most common baseline models for time series forecasting, the naïve model (also called the [naïve forecast](https://otexts.com/fpp3/simple-methods.html#na%C3%AFve-method)), requires no training at all.

That's because all the naïve model does is use the previous timestep value to predict the next timestep value.

The formula looks like this:

$$\hat{y}_{t} = y_{t-1}$$ 

In English: 
> The prediction at timestep `t` (y-hat) is equal to the value at timestep `t-1` (the previous timestep).


In an open system (like a stock market or crypto market), you'll often find beating the naïve forecast with *any* kind of model is quite hard.

> 🔑 **Note:** For the sake of this notebook, an **open system** is a system where inputs and outputs can freely flow, such as a market (stock or crypto). Where as, a **closed system** the inputs and outputs are contained within the system (like a poker game with your buddies, you know the buy in and you know how much the winner can get). Time series forecasting in **open systems** is generally quite poor.

ANALIZAR

In [18]:
def make_future_forecast_generalized(model, initial_window, into_future, horizon, window_size=32, verbose=False) -> np.ndarray:
    """
    Generate future predictions using a sliding window for models that output multiple steps at once.
    Returns predictions as a 2D array of shape (num_forecasts, horizon).
    """
    future_forecast = []
    last_window = initial_window.copy()
    
    steps = int(np.ceil(into_future / horizon))

    for _ in range(steps):
        future_pred = model.predict(tf.expand_dims(last_window, axis=0), verbose=0)  # shape: (1, horizon)
        future_pred = tf.squeeze(future_pred).numpy()  # shape: (horizon,)
        
        if verbose:
            print(f"Predicting on: \n{last_window} -> Prediction: {future_pred}\n")

        future_forecast.append(future_pred)  # Append as row, not flattened

        # Update window
        last_window = np.append(last_window, future_pred)[-window_size:]

    # Return as (num_steps, horizon), trim excess if needed
    forecast_array = np.array(future_forecast)  # shape: (steps, horizon)
    
    # If exact match is needed with test_labels shape:
    total_needed = (into_future // horizon)
    return forecast_array[:total_needed]


In [20]:
#into_future = len(test_labels) * horizon
future_forecast_3 = make_future_forecast_generalized(model=model_3,
                                       initial_window=test_windows[0],
                                       into_future=len(test_labels)*HORIZON, # NEW
                                       horizon=HORIZON,            
                                       window_size=WINDOW_SIZE)

# HACER LO DEL HORIZONTE MULTIPLE EN MI BACKGROUND CON DATIS SIMPLES Y LISTO! 
                                       

NameError: name 'model_3' is not defined